In [ ]:
!pip install -q transformers accelerate requests bitsandbytes
!pip install --upgrade transformers

In [ ]:
import requests
import torch
import json
from datetime import datetime
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

REPO = "Mintplex-Labs/anything-llm"
MODEL_NAME = "microsoft/Phi-3.5-mini-instruct"


In [ ]:
def coletar_dados():
    releases = requests.get(
        f"https://api.github.com/repos/{REPO}/releases?per_page=10"
    ).json()

    commits = requests.get(
        f"https://api.github.com/repos/{REPO}/commits?per_page=30"
    ).json()

    return releases, commits


def salvar_json(nome_arquivo, conteudo):
    with open(nome_arquivo, "w", encoding="utf-8") as f:
        json.dump(conteudo, f, indent=2, ensure_ascii=False)

In [ ]:
releases, commits = coletar_dados()

salvar_json(
    "dados_anythingllm.json",
    {
        "repositorio": REPO,
        "coletado_em": datetime.now().isoformat(),
        "releases": releases,
        "commits": commits
    }
)


In [ ]:
release_info = []
for r in releases:
    tag = r.get("tag_name", "N/A")
    date = r.get("published_at", "N/A")
    release_info.append(f"{tag} - {date}")

prompt = f"""
Você é um especialista em Engenharia de Software.

Com base APENAS nos dados abaixo, identifique a ESTRATÉGIA DE RELEASES
do projeto AnythingLLM.

DADOS DE RELEASES (tag - data):
{chr(10).join(release_info)}

TOTAL DE RELEASES ANALISADAS: {len(releases)}
TOTAL DE COMMITS RECENTES ANALISADOS: {len(commits)}

CLASSIFIQUE A ESTRATÉGIA COMO UMA DAS SEGUINTES:
- Rapid Releases
- Release Train
- LTS + Current
- Ad hoc / Irregular

RESPONDA NO FORMATO:
1. Estratégia identificada:
2. Frequência observada:
3. Uso de versionamento semântico (Sim/Não):
4. Justificativa baseada nos dados:
"""

salvar_json(
    "prompt_anythingllm.json",
    {
        "modelo": MODEL_NAME,
        "prompt": prompt
    }
)


In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)


In [ ]:
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=2048
)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=400,
        temperature=0.1,
        do_sample=False
    )

resposta = tokenizer.decode(
    output[0],
    skip_special_tokens=True
)

print("===== RESULTADO FINAL =====\n")
print(resposta)


In [ ]:
salvar_json(
    "resultado_analise_anythingllm.json",
    {
        "repositorio": REPO,
        "modelo": MODEL_NAME,
        "data_analise": datetime.now().isoformat(),
        "resultado": resposta
    }
)

print("\nArquivos gerados:")
print("- dados_anythingllm.json")
print("- prompt_anythingllm.json")
print("- resultado_analise_anythingllm.json")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')